<a href="https://colab.research.google.com/github/ArshT/Mini-Projects/blob/master/Poster_Genre_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
###Mounting the Drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd '/content/drive/My Drive'

/content/drive/My Drive


In [0]:
###Import the CSV File
import pandas as pd

data = pd.read_csv('movie_poster.csv')
data.head()

,tid,title,url,image_url,year,action,scifi,drama,romance
0,tt0120737,The Lord of the Rings: The Fellowship of the R...,http://www.imdb.com/title/tt0120737/,https://m.media-amazon.com/images/M/MV5BN2EyZj...,2001,True,False,False,False
1,tt0167260,The Lord of the Rings: The Return of the King,http://www.imdb.com/title/tt0167260/,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,2003,True,False,False,False
2,tt0167261,The Lord of the Rings: The Two Towers,http://www.imdb.com/title/tt0167261/,https://m.media-amazon.com/images/M/MV5BNGE5Mz...,2002,True,False,False,False
3,tt0169102,Lagaan: Once Upon a Time in India,http://www.imdb.com/title/tt0169102/,https://m.media-amazon.com/images/M/MV5BNDYxNW...,2001,False,False,True,False
4,tt0211915,Amélie,http://www.imdb.com/title/tt0211915/,https://m.media-amazon.com/images/M/MV5BNDg4Nj...,2001,False,False,False,True


In [0]:
###Loading the Posters
import glob

local_download_path = '/content/drive/My Drive/posters'
images = glob.glob(local_download_path + '/*.jpg')

In [0]:
###Number of Posters

n_posters = len(images)
n_posters

2438

In [0]:
###Importing the important Libraries

import numpy as np
import pandas as pd
import glob
import scipy.misc
import imageio
import skimage
from tqdm import tqdm
import requests  
import re
from bs4 import BeautifulSoup  
from urllib.request import urlretrieve
import ast 
import matplotlib.pyplot as plt 

In [0]:
###Function for getting the Filename

def get_id(filename):
  index_s = 32
  index_f = filename.rfind(".jpg")
  
  return filename[index_s:index_f]

In [0]:
###Function for Pre-Processing
from skimage.transform import resize

def preprocess(img,size=(150,101,3)):
  img = skimage.transform.resize(img, size)
  img = img.astype(np.float32)
  img = (img / 127.5) - 1.

  return img

In [0]:
###Converting an Image into array and storing the arrays in a dict

from matplotlib import image
img_dict = {}
counter = 0

for fn in images:
  try:
    counter+=1
    if counter%100 == 0:
      print(counter,get_id(fn))
    img = image.imread(fn)
    if counter%100 == 0:
      print(img.shape)
    img = preprocess(img)
    if counter%100 == 0:
      print(img.shape)
    img_dict[get_id(fn)] = img
    if counter%100 == 0:
      print("WORKS")
      print("")
  except:
    print('DOESNT WORK')

print("Done")

100 tt1546036
(500, 333, 3)
(150, 101, 3)
WORKS

200 tt1951265
(2048, 1347, 3)
(150, 101, 3)
WORKS

300 tt0758053
(666, 449, 3)
(150, 101, 3)
WORKS

400 tt0857190
(2048, 1382, 3)
(150, 101, 3)
WORKS

500 tt1001526
(2048, 1311, 3)
(150, 101, 3)
WORKS

600 tt1135985
(1000, 674, 3)
(150, 101, 3)
WORKS



/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2731: DecompressionBombWarning: Image size (94235400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


700 tt1268987
(2048, 1458, 3)
(150, 101, 3)
WORKS

800 tt1419791
(2941, 2239, 3)
(150, 101, 3)
WORKS

900 tt1568341
(8726, 6313, 3)
(150, 101, 3)
WORKS

1000 tt1701991
(367, 653, 3)
(150, 101, 3)
WORKS

1100 tt1533749
(2048, 1382, 3)
(150, 101, 3)
WORKS

1200 tt0246677
(668, 450, 3)
(150, 101, 3)
WORKS

1300 tt0303461
(882, 600, 3)
(150, 101, 3)
WORKS

1400 tt0362192
(2048, 1447, 4)
(150, 101, 3)
WORKS

1500 tt0406375
(1440, 960, 3)
(150, 101, 3)
WORKS

1600 tt0460627
(1500, 971, 3)
(150, 101, 3)
WORKS

1700 tt0765592
(375, 500, 3)
(150, 101, 3)
WORKS

1800 tt0908836
(375, 500, 3)
(150, 101, 3)
WORKS

1900 tt1081935
(683, 480, 3)
(150, 101, 3)
WORKS

2000 tt1270798
(918, 620, 3)
(150, 101, 3)
WORKS

2100 tt0705262
(750, 1000, 3)
(150, 101, 3)
WORKS

2200 tt1228705
(755, 509, 3)
(150, 101, 3)
WORKS

2300 tt0404203
(667, 450, 3)
(150, 101, 3)
WORKS

2400 tt0414993
(2048, 1387, 3)
(150, 101, 3)
WORKS

Done


In [0]:
###Function for Creating a Dataset

def DataSet(img_dict,data):
  m = len(img_dict)
  X = np.zeros((m,150,101,3))
  Y = np.zeros((m,4))

  i=0
  for tid in data['tid']:
    try:
      X[i,:,:,:] = img_dict[tid]
      labels = data[data['tid'] == tid][['action','scifi','drama','romance']]*1
      Y[i,:] = np.array(labels)
      i+=1
    except:
      continue
    
  return X[:i,:,:,:],Y[:i,:]

In [0]:
###Making the Dataset

X,Y = DataSet(img_dict,data)

In [0]:
print(X.shape)
print("")
print(Y.shape)

(2436, 150, 101, 3)

(2436, 4)


In [0]:
###Model

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(150,101, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='sigmoid'))

In [0]:
###Splitting the Datset

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1948, 150, 101, 3)
(1948, 4)
(488, 150, 101, 3)
(488, 4)


In [0]:
###Compiling the Model

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
###Training the Model

model.fit(X_train,Y_train, batch_size=16, epochs=10,
          verbose=1, validation_split=0.1)

Train on 1753 samples, validate on 195 samples
Epoch 1/10
1753/1753 [==============================] - 4s 2ms/step - loss: 0.5486 - acc: 0.7648 - val_loss: 0.5162 - val_acc: 0.7833
Epoch 2/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5180 - acc: 0.7857 - val_loss: 0.5115 - val_acc: 0.7833
Epoch 3/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5145 - acc: 0.7864 - val_loss: 0.5088 - val_acc: 0.7833
Epoch 4/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5117 - acc: 0.7867 - val_loss: 0.5089 - val_acc: 0.7833
Epoch 5/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5098 - acc: 0.7867 - val_loss: 0.5096 - val_acc: 0.7833
Epoch 6/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5076 - acc: 0.7867 - val_loss: 0.5090 - val_acc: 0.7833
Epoch 7/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.5064 - acc: 0.7867 - val_loss: 0.5102 - val_acc: 0.7833
Epoch 8/10
1753

In [0]:
###Testing the Model

score = model.evaluate(X_test, Y_test, verbose=1)
score

488/488 [==============================] - 0s 915us/step


[0.509299244059891, 0.7827868852459017]